In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import JSONLoader
from langchain.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
from langchain_huggingface import HuggingFaceEmbeddings

load_dotenv()

file_path = "FAQ.json"

# Load the JSON File as Documents
loader = JSONLoader(file_path=file_path, jq_schema=".[]", text_content=False)
documents = loader.load()

# Initialize an Open-Source Embedding Model (using SentenceTransformers)
model_name = "all-MiniLM-L6-v2"  # Lightweight and free
# Create a HuggingFaceEmbeddings instance which internally uses SentenceTransformers
embeddings = HuggingFaceEmbeddings(model_name=model_name)

# Create a FAISS Vector Store from the Documents
faiss_db = FAISS.from_documents(documents, embeddings)

# (Optional) Persist the FAISS index locally for later use.
faiss_db.save_local("data/faiss_index")

In [ ]:
# Example query text
query = "What delivery options?"

# Perform a similarity search
results = faiss_db.similarity_search(query, )

# Print out the results
for result in results:
    print(result)  # or result.metadata, depending on what you stored

### If the Computer is restarted then you can load local

In [ ]:
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

# Load the persisted FAISS index with dangerous deserialization allowed
faiss_db = FAISS.load_local("data/faiss_index", embeddings, allow_dangerous_deserialization=True)

query = "how to create something?"
results = faiss_db.similarity_search(query)

for result in results:
    print(result.page_content)

### Code to add other json files (i.e., new data) into this existing vector db

In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import JSONLoader
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

# Load environment variables
load_dotenv()

# Define paths for the new JSON files
new_files = [
    "privacy-policy.json",
    "common-questions.json"
]

# Initialize your embedding model (ensure it's the same as used before)
model_name = "all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

# Load the existing FAISS index from disk, allowing dangerous deserialization if needed
faiss_db = FAISS.load_local("data/faiss_index", embeddings, allow_dangerous_deserialization=True)

# Loop over each new JSON file, load documents and add them to the index
for file_path in new_files:
    # Load the JSON file as documents
    loader = JSONLoader(file_path=file_path, jq_schema=".[]", text_content=False)
    new_documents = loader.load()
    
    # Add the new documents to the existing FAISS index
    faiss_db.add_documents(new_documents)

# Optionally, persist the updated FAISS index
faiss_db.save_local("data/faiss_index")

In [ ]:
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

# Load the persisted FAISS index with dangerous deserialization allowed
faiss_db = FAISS.load_local("data/faiss_index", embeddings, allow_dangerous_deserialization=True)

query = "troubleshoot"
results = faiss_db.similarity_search(query)

for result in results:
    print(result.page_content)

---

In [ ]:
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

def query_faiss(user_query, index_path="data/faiss_index", top_k=5):
    """
    Given a user query, this function loads the persisted FAISS index,
    performs a similarity search, and returns the top_k matching documents.
    
    Args:
        user_query (str): The input query from the user.
        index_path (str): The path where the FAISS index is saved.
        top_k (int): Number of similar documents to return.
        
    Returns:
        list: A list of the top_k matching document objects.
    """
    # Initialize the embedding model (should match the one used to build the index)
    model_name = "all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    
    # Load the persisted FAISS index (allow dangerous deserialization if needed)
    faiss_db = FAISS.load_local(index_path, embeddings, allow_dangerous_deserialization=True)
    
    # Perform similarity search with the given query
    results = faiss_db.similarity_search(user_query, k=top_k)
    
    return results

In [ ]:
# Example usage:
if __name__ == "__main__":
    query = input("Enter your query: ")
    similar_docs = query_faiss(query)
    
    print("\nTop matching documents:")
    for doc in similar_docs:
        print("-----")
        print(doc.page_content)  # or use doc.metadata if that's more appropriate

---

In [ ]:
import os
from dotenv import load_dotenv
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamlitCallbackHandler  # example callback for streaming output
from langchain.schema import HumanMessage, SystemMessage

# Load environment variables (make sure OPENAI_API_KEY is set)
load_dotenv()


def construct_rag_prompt(query, retrieved_docs):
    """
    Build a prompt that includes retrieved document content and the user's query.
    """
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])
    prompt = (
        "You are a helpful assistant that uses the following context to answer the question.\n\n"
        "Context:\n"
        f"{context}\n\n"
        "Question:\n"
        f"{query}\n\n"
        "Note: If the 'Question' the user asks is not present in the 'Context', then you will respond with 'Unfortunately we do not have answer for that."
    )
    return prompt

In [ ]:
from openai import OpenAI

# Get user's query
user_query = input("Enter your query: ")

# Step 1: Retrieve similar documents using the FAISS index
retrieved_docs = query_faiss(user_query, index_path="data/faiss_index", top_k=3)

# Step 2: Construct the RAG prompt
prompt = construct_rag_prompt(user_query, retrieved_docs)
print("\n\n--- RAG Prompt ---\n")
print(prompt)
print("\n\n--- Streaming Response ---\n")

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

stream = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant that answers questions using provided context."},
        {"role": "user", "content": prompt}
    ],
    stream=True,  # Enable streaming mode
    temperature=0.7,
    max_tokens=1000,
)

answer = ""

for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="")